# download necessary files

In [1]:
#download necessary files
import nltk
nltk.download('punkt')
nltk.download('reuters')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aggel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\aggel\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!


True

## Initializing Reuters Corpus + Building Vocab

In [2]:
#query (i)a Corpus and vocabualry preprocesing
#------------------

from collections import defaultdict
from tqdm import tqdm

# Load train and test corpus
corpus = nltk.corpus.reuters.sents()[:3000]  # Sample train_corpus

test_corpus = nltk.corpus.reuters.sents()[18000:18600]  # Sample test_corpus

development_corpus = nltk.corpus.reuters.sents()[19000:19300]  # Sample test_corpus

def build_vocabulary(corpus, min_frequency):
    word_counts = defaultdict(int)
    for sentence in corpus:
        for word in sentence:
            word_counts[word] += 1
    vocabulary = {word for word, count in word_counts.items() if count >= min_frequency}
    vocabulary.add('*UNK*')
    vocabulary.add('*end*')
    return vocabulary

# # 2. Minimum Count Threshold for OOV words
min_frequency = 10

vocabulary = build_vocabulary(corpus, min_frequency)

vocabulary = list(vocabulary)

modified_corpus = [list(sentence) for sentence in corpus]

modified_test_corpus = [list(sentence) for sentence in test_corpus]

modified_development_corpus = [list(sentence) for sentence in development_corpus]

# Initialize the counter for *UNK* token
unk_count = 0

# Replacing OOV words with the token *UNK* in train corpus
for sentence in tqdm(range(len(modified_corpus)), desc="Replace OOV with '*UNK*' in train_corpus"):
    for word_idx in range(len(modified_corpus[sentence])):
        word = modified_corpus[sentence][word_idx]
        if word not in vocabulary:
            modified_corpus[sentence][word_idx] = '*UNK*'
            unk_count += 1

corpus = modified_corpus

# Replacing OOV words with the token *UNK* in test_corpus
for sentence in tqdm(range(len(modified_test_corpus)), desc="Replace OOV with '*UNK*' in test_corpus"):
    for word_idx in range(len(modified_test_corpus[sentence])):
        word = modified_test_corpus[sentence][word_idx]
        if word not in vocabulary:
            modified_test_corpus[sentence][word_idx] = '*UNK*'

test_corpus = modified_test_corpus


# Replacing OOV words with the token *UNK* in development_corpus
for sentence in tqdm(range(len(modified_development_corpus)), desc="Replace OOV with '*UNK*' in develpment_corpus"):
    for word_idx in range(len(modified_development_corpus[sentence])):
        word = modified_development_corpus[sentence][word_idx]
        if word not in vocabulary:
            modified_development_corpus[sentence][word_idx] = '*UNK*'

development_corpus = modified_development_corpus


print(len(vocabulary))
print(unk_count)

Replace OOV with '*UNK*' in develpment_corpus: 100%|███████████████████████████████| 300/300 [00:00<00:00, 4503.40it/s]

1187
18961


## Creation of Bigram & Trigram

In [3]:
#query (i)b Creating the models
#------------------
import numpy as np

# Define a function to build a bigram model with Kneser-Ney smoothing
def build_bigram_model_kneser_ney(corpus, vocabulary, discount):
    # Create a copy of the vocabulary and add '*start*' to it
    temp_vocabulary = vocabulary.copy()
    temp_vocabulary.append('*start*')
    
    # Create a 2D array to represent the bigram model
    bigram_model = [[0. for _ in temp_vocabulary] for _ in temp_vocabulary]
    
    # Initialize count dictionaries for Kneser-Ney smoothing
    unigram_count = [0.0] * len(temp_vocabulary)
    continuation_count = [0.0] * len(temp_vocabulary)
    
    # Process each sentence in the corpus using tqdm for progress tracking
    for sentence in tqdm(corpus, desc="Processing sentences"):
        # Add '*start*' and '*end*' tokens to the sentence
        sentence = ['*start*'] + sentence + ['*end*']
        for i in range(len(sentence) - 1):
            word1 = sentence[i]
            word2 = sentence[i+1]
            
            # Get the indices of the two words in the vocabulary
            word1_idx = temp_vocabulary.index(word1)
            word2_idx = temp_vocabulary.index(word2)

            # Update the bigram model and count dictionaries
            bigram_model[word1_idx][word2_idx] += 1.0
            unigram_count[word1_idx] += 1.0
            continuation_count[word2_idx] += 1.0

    # smaller discount moves less probability mass to lower grams making the smoothing proccess weeker         
    # Apply Kneser-Ney smoothing to the bigram model
    for word1 in range(len(temp_vocabulary)):
        for word2 in range(len(temp_vocabulary)):

            p_continuation = continuation_count[word2] / sum(continuation_count)
            
            # Handle division by zero and apply smoothing
            lambda_value = (discount / (unigram_count[word1] + unk_count)) * len(continuation_count)
            
            p_continuation = max(p_continuation - discount, 0)
            
            # Smooth the bigram model
            prob = (max(bigram_model[word1][word2] - discount, 0) / (unigram_count[word1] + unk_count)) + lambda_value * p_continuation

            bigram_model[word1][word2] = prob

    # Return the smoothed bigram model
    return bigram_model

# smaller discount moves less probability mass to lower grams making the smoothing proccess weeker 
discount = 0.0005

# Print a message to indicate that we are building the bigram model
print("Building Bigram model ...")

# Call the function to build the bigram model and store the result in bigram_model
bigram_model = build_bigram_model_kneser_ney(corpus, vocabulary, discount)

# Define a function to build a trigram model with Kneser-Ney smoothing
def build_trigram_model_kneser_ney(corpus, vocabulary, discount):
    # Create a copy of the vocabulary and add '*start1*' and '*start2*' to it
    temp_voc = vocabulary.copy()
    temp_voc.append('*start1*')
    temp_voc.append('*start2*')
    vocab_size = len(temp_voc)
    
    # Create a 2D numpy array to represent the trigram model
    trigram_model = np.zeros((vocab_size ** 2, vocab_size), dtype=np.float64)

    # Initialize count dictionaries for Kneser-Ney smoothing
    unigram_count = np.zeros(vocab_size, dtype=np.float64)
    bigram_count = np.zeros((vocab_size, vocab_size), dtype=np.float64)
    continuation_count = np.zeros(vocab_size, dtype=np.float64)

    # Initialize a dictionary to store trigram counts
    trigram_counts = {}

    # Process each sentence in the corpus using tqdm for progress tracking
    for sentence in tqdm(corpus, desc="Processing sentences"):
        # Add '*start1*', '*start2*', and '*end*' tokens to the sentence
        sentence = ['*start1*'] + ['*start2*'] + sentence + ['*end*']
        for i in range(len(sentence) - 2):
            word1, word2, word3 = sentence[i], sentence[i+1], sentence[i+2]

            # Get the indices of the three words in the vocabulary
            word1_idx, word2_idx, word3_idx = temp_voc.index(word1), temp_voc.index(word2), temp_voc.index(word3)

            # Calculate a flat index for the trigram model
            word1_flat = word1_idx * vocab_size + word2_idx

            # Update the trigram model and count dictionaries
            trigram_model[word1_flat, word3_idx] += 1.0
            unigram_count[word1_idx] += 1.0
            bigram_count[word1_idx, word2_idx] += 1.0
            continuation_count[word3_idx] += 1.0

            # Store trigram counts in the dictionary
            trigram = (word1, word2, word3)
            trigram_counts[trigram] = trigram_counts.get(trigram, 0) + 1

    # Define threshold and discount values for pruning and smoothing
    threshold = 0.0000000000000001

    # Create a set to store pruned trigrams
    pruned_trigrams = set()

    # Prune low-count trigrams based on the threshold
    total_trigrams = len(trigram_counts)
    for trigram, count in trigram_counts.items():
        if float(count) / total_trigrams >= threshold:
            pruned_trigrams.add(trigram)

    # Apply Kneser-Ney smoothing using only pruned trigrams
    for word1 in tqdm(range(vocab_size), desc="Kneser-Ney smoothing Iterations"):
        for word2 in range(vocab_size):
            for word3 in range(vocab_size):
                if (word1, word2, word3) in pruned_trigrams:
                    p_continuation = continuation_count[word3] / sum(continuation_count)

                    # Handle division by zero for unigram_count and bigram_count, and apply smoothing
                    lambda_value = (discount / (unigram_count[word1] + unk_count)) * len(continuation_count)

                    p_continuation = max(p_continuation - discount, 0)
                    
                    # Smooth the trigram model
                    prob = (max(trigram_model[word1_flat, word3_idx] - discount, 0) / (bigram_count[word1_idx, word2_idx] + unk_count)) + lambda_value * p_continuation

                    trigram_model[word1_flat, word3_idx] = prob

    # Return the smoothed trigram model
    return trigram_model

# Print a message to indicate that we are building the trigram model
print("Building Trigram model ...")

# smaller discount moves less probability mass to lower grams making the smoothing proccess weeker 
discount = 0.0005

# Call the function to build the trigram model and store the result in trigram_model
trigram_model = build_trigram_model_kneser_ney(corpus, vocabulary, discount)

# Print a message to indicate that the model building is finished
print("Finish building")

Building Bigram model ...


Processing sentences: 100%|██████████████████████████████████████████████████████| 3000/3000 [00:01<00:00, 2095.58it/s]


Building Trigram model ...


Kneser-Ney smoothing Iterations: 100%|█████████████████████████████████████████████| 1189/1189 [02:18<00:00,  8.61it/s]

Finish building


## Cross-Entropy & Perplexity Models evaluation

In [4]:
#query (ii) Models Evaluation
#------------------

from math import log

# Define a function to calculate cross-entropy and perplexity for a bigram model
def calculate_cross_entropy_and_perplexity_bigram(model, test_corpus):
    total_log_probability = 0.0
    total_bigrams = 0
    temp_vocabulary = vocabulary.copy()
    temp_vocabulary.append('*start*')

    # Iterate through the test corpus
    for sentence in test_corpus:
        sentence = ['*start*'] + sentence + ['*end*']

        for i in range(1, len(sentence) - 1):
            current_word = sentence[i]
            next_word = sentence[i + 1] 

            # Get the probability of the bigram from the model
            probability = model[temp_vocabulary.index(current_word)][temp_vocabulary.index(next_word)]

            # Handle zero probabilities by replacing with a small value
            epsilon = 1e-10  # Small positive value
            if probability == 0:
                probability = epsilon

            total_log_probability += log(probability)
            total_bigrams += 1

    # Calculate cross-entropy and perplexity
    cross_entropy = -total_log_probability / total_bigrams
    perplexity = 2 ** cross_entropy
    return cross_entropy, perplexity

# Define a function to calculate cross-entropy and perplexity for a trigram model
def calculate_cross_entropy_and_perplexity_trigram(model, test_corpus):
    total_log_probability = 0.0
    total_trigrams = 0
    temp_vocabulary = vocabulary.copy()
    temp_vocabulary.append('*start1*')
    temp_vocabulary.append('*start2*')

    # Iterate through the test corpus
    for sentence in test_corpus:
        sentence = ['*start1*'] + ['*start2*'] + sentence + ['*end*']

        for i in range(2, len(sentence) - 2):
            current_word = sentence[i]
            next_word = sentence[i + 1] 
            next_next_word = sentence[i + 2] 

            # Get the probability of the trigram from the model
            probability = model[temp_vocabulary.index(current_word) * len(temp_vocabulary) + temp_vocabulary.index(next_word)][temp_vocabulary.index(next_next_word)]

            # Handle zero probabilities by replacing with a small epsilon value
            epsilon = 1e-10  # Small positive value
            if probability == 0:
                probability = epsilon

            total_log_probability += log(probability)
            total_trigrams += 1

    # Calculate cross-entropy and perplexity
    cross_entropy = -total_log_probability / total_trigrams
    perplexity = 2 ** cross_entropy
    return cross_entropy, perplexity

# Calculate cross-entropy and perplexity for the bigram model
cross_entropy_bigram, perplexity_bigram = calculate_cross_entropy_and_perplexity_bigram(bigram_model, test_corpus)

# Calculate cross-entropy and perplexity for the trigram model
cross_entropy_trigram, perplexity_trigram = calculate_cross_entropy_and_perplexity_trigram(trigram_model, test_corpus)

# Print the cross-entropy and perplexity values for both models
print('Bigram Cross-Entropy:', cross_entropy_bigram)
print('Bigram Perplexity:', perplexity_bigram)
print('Trigram Cross-Entropy:', cross_entropy_trigram)
print('Trigram Perplexity:', perplexity_trigram)

Bigram Cross-Entropy: 7.969421458000477
Bigram Perplexity: 250.63106927017245
Trigram Cross-Entropy: 8.392648789071439
Trigram Perplexity: 336.07718322906413


## Finding the best Discount Hyperparameter


In [68]:

#Testing different values for discount to choose which one goes better in the development_corpus 

# Define the list of discount values to test for the bigram and trigram models

bigram_discount_values = [0.0005, 0.0015, 0.0025, 0.0035, 0.0045, 0.0055, 0.0065, 0.0075]

trigram_discount_values = [0.0005, 0.0015, 0.0025, 0.0035, 0.0045, 0.0055, 0.0065, 0.0075]

best_bigram_perplexity = float('inf')
best_bigram_discount = None

# Loop for the Bigram model
for bigram_discount in bigram_discount_values:
    # Build the bigram model with the current discount
    bigram_model = build_bigram_model_kneser_ney(corpus, vocabulary, bigram_discount)

    # Calculate perplexity for the bigram model
    _, perplexity_bigram = calculate_cross_entropy_and_perplexity_bigram(bigram_model, development_corpus)

    # Choose the best discount for the bigram model
    if perplexity_bigram < best_bigram_perplexity:
        best_bigram_perplexity = perplexity_bigram
        best_bigram_discount = bigram_discount

print("Best discount for Bigram model:", best_bigram_discount)

best_trigram_perplexity = float('inf')
best_trigram_discount = None

# Loop for the Trigram model
for trigram_discount in trigram_discount_values:
    # Build the trigram model with the current discount
    trigram_model = build_trigram_model_kneser_ney(corpus, vocabulary, trigram_discount)

    # Calculate perplexity for the trigram model
    cros_en_tr, perplexity_trigram = calculate_cross_entropy_and_perplexity_trigram(trigram_model, development_corpus)

    # Choose the best discount for the trigram model
    if perplexity_trigram < best_trigram_perplexity:
        best_trigram_perplexity = perplexity_trigram
        best_trigram_discount = trigram_discount

print("Best discount for Trigram model:", best_trigram_discount)

Processing sentences: 100%|██████████████████████████████████████████████████████| 3000/3000 [00:01<00:00, 2134.18it/s]


Best discount for Bigram model: 0.0005


Kneser-Ney smoothing Iterations: 100%|█████████████████████████████████████████████| 1189/1189 [02:18<00:00,  8.56it/s]


Best discount for Trigram model: 0.0005


## Sentense Completion using the Bigram Model

In [15]:
#query (iii)a bigram sentence completion
#------------------
import random
# Function to generate the next word using the bigram model
def generate_next_word_bigram(sentence, model, vocabulary):
    # Get the last word in the sentence
    last_word = sentence[-1]
    # Find the index of the last word in the vocabulary or use the index of '*UNK*' if it's not in the vocabulary
    last_word_index = vocabulary.index(last_word) if last_word in vocabulary else vocabulary.index('*UNK*')
    # Get the probabilities of the next word given the last word
    next_word_probs = model[last_word_index]

    # Get the top candidates based on probabilities
    top_candidates = sorted(range(len(next_word_probs)), key=lambda word_idx: next_word_probs[word_idx], reverse=True)[:4]

    # Randomly select the next word from the top candidates
    next_word_index = random.choice(top_candidates)
    next_word = vocabulary[next_word_index]
    return next_word

# Function to generate completions for an incomplete sentence using the bigram model and beam search
def generate_bigram_completion(incomplete_sentence, model, vocabulary):
    completions = []

    # Seed initial completions with the incomplete sentence
    initial_completions = [incomplete_sentence[:]]

    for _ in range(300):  # Limit the number of generated words
        new_completions = []

        for completion in initial_completions:
            # Generate the next word for each completion
            next_word = generate_next_word_bigram(completion, model, vocabulary)
            new_completion = completion + [next_word]
            new_completions.append(new_completion)

            # If the next word is the end token, add the completion to the list of completions
            if next_word == '*end*':
                completions.append(new_completion[1:])  # Remove the 'start' token
            else:
                initial_completions = new_completions
                
    return [' '.join(completion) for completion in completions]

# Function to calculate the log probability of a sentence using the bigram model
def calculate_log_sentence_probability(sentence, model, vocabulary):
    log_probability = 0.0
    for i in range(1, len(sentence)):
        prev_word = sentence[i - 1]
        word = sentence[i]
        if prev_word in vocabulary and word in vocabulary:
            prev_word_idx = vocabulary.index(prev_word)
            word_idx = vocabulary.index(word)
            probability = model[prev_word_idx][word_idx]
            # Handle zero probabilities by replacing with a small value
            epsilon = 1e-10  # Small positive value
            if probability == 0:
                probability = epsilon
            
            log_probability += log(probability)
    return log_probability

# Function to choose the best completion from a list of completions
def choose_best_completion(completions, model, vocabulary):
    best_completion = None
    best_log_probability = float('-inf')
    for completion in completions:
        log_probability = calculate_log_sentence_probability(completion, model, vocabulary)
        if log_probability > best_log_probability:
            best_completion = completion
            best_log_probability = log_probability
    return best_completion

# Set a random seed for reproducibility
random.seed(400)
# Example incomplete sentence
incomplete_sentence = ['*start*', 'I', 'would', 'like', 'to', 'commend', 'the']

# Generate completions using the bigram model
bigram_completions = generate_bigram_completion(incomplete_sentence, bigram_model, vocabulary)

# Choose the best completion based on log probability
best_completion = choose_best_completion(bigram_completions, bigram_model, vocabulary)

print("Best Bigram Completion:", best_completion)

# Another example incomplete sentence
incomplete_sentence2 = ['*start*', 'According', 'to', 'the', 'report']

# Generate completions using the bigram model for the second incomplete sentence
bigram_completions = generate_bigram_completion(incomplete_sentence2, bigram_model, vocabulary)

# Choose the best completion based on log probability for the second sentence
best_completion = choose_best_completion(bigram_completions, bigram_model, vocabulary)

print("Best Bigram Completion for the second sentence:", best_completion)

# Another example incomplete sentence
incomplete_sentence3 = ['*start*', 'In', 'my', 'opinion']

# Generate completions using the bigram model for the second incomplete sentence
bigram_completions = generate_bigram_completion(incomplete_sentence3, bigram_model, vocabulary)

# Choose the best completion based on log probability for the second sentence
best_completion = choose_best_completion(bigram_completions, bigram_model, vocabulary)

print("Best Bigram Completion for the second sentence:", best_completion)

Best Bigram Completion: I would like to commend the company *UNK* *UNK* of its domestic markets to be used *UNK* of the *UNK* , *UNK* of a share *UNK* , *UNK* *UNK* *UNK* . *end*
Best Bigram Completion for the second sentence: According to the report . *UNK* of its 1986 / 86 crop at the company . *UNK* , which is *UNK* . *UNK* . S . *UNK* *UNK* . *end*
Best Bigram Completion for the second sentence: In my opinion of its domestic markets , *UNK* . *end*


## Sentense Completion using the Trigram Model

In [16]:
#query (iii)b trigram sentence completion
#------------------

import random
random.seed(30000)
from math import log

random.seed(30000)

# Import the log function from the math module
from math import log

# Function to generate the next word using the trigram model
def generate_next_word_trigram(sentence, model, vocabulary):
    # Get the last two words in the sentence
    last_two_words = sentence[-2:]

    # Check if the last two words are in the vocabulary, if not, replace with '*UNK*'
    last_two_words[0] = last_two_words[0] if last_two_words[0] in vocabulary else '*UNK*'
    last_two_words[1] = last_two_words[1] if last_two_words[1] in vocabulary else '*UNK*'

    # Get the probabilities of the next word given the last two words
    next_word_probs = model[vocabulary.index(last_two_words[0]) + vocabulary.index(last_two_words[1])]

    # Find the maximum probability among all words
    max_prob = max(next_word_probs)

    # Get all words with the maximum probability
    max_prob_words = [vocabulary[i] for i, prob in enumerate(next_word_probs) if prob == max_prob]

    # Randomly choose one word from the words with maximum probability
    next_word = random.choice(max_prob_words)
    return next_word

# Function to generate the best completion for an incomplete sentence using the trigram model
def generate_best_trigram_completion(incomplete_sentence, model, vocabulary):
    best_completion = incomplete_sentence[:]
    temp_vocabulary = vocabulary.copy()
    temp_vocabulary.append('*start1*')
    temp_vocabulary.append('*start2*')

    for _ in range(300):  # Limit the number of generated words
        next_word = generate_next_word_trigram(best_completion, model, temp_vocabulary)
        best_completion.append(next_word)

        # If the next word is the end token, return the best completion (excluding 'start' tokens)
        if next_word == '*end*':
            return ' '.join(best_completion[2:])

    # If no end token is encountered, return the best completion (excluding 'start' tokens)
    return ' '.join(best_completion[2:])

# Example incomplete sentence
incomplete_sentence31 = ['*start1*', '*start2*', 'I', 'would', 'like', 'to', 'commend', 'the']

# Generate the best completion for the first incomplete sentence
best_completion31 = generate_best_trigram_completion(incomplete_sentence31, trigram_model, vocabulary)

print("Best Trigram Completion (incomplete_sentence31):", best_completion31)

incomplete_sentence32 = ['*start1*', '*start2*', 'According', 'to', 'the', 'report']

# Generate the best completion for the second incomplete sentence
best_completion32 = generate_best_trigram_completion(incomplete_sentence32, trigram_model, vocabulary)

print("Best Trigram Completion (incomplete_sentence32):", best_completion32)

incomplete_sentence33 = ['*start1*', '*start2*', 'In', 'my', 'opinion']

# Generate the best completion for the second incomplete sentence
best_completion33 = generate_best_trigram_completion(incomplete_sentence33, trigram_model, vocabulary)

print("Best Trigram Completion (incomplete_sentence32):", best_completion33)

Best Trigram Completion (incomplete_sentence31): I would like to commend the AT report Clevite eight question 84 agreement cost discount sugar net 11 REUTER company pre given General 27 members open traders NOTE did of Inc billion 87 INC around X Kong NEW imports third good sold European shipment 138 Avg de loans too Exports Louvre The FROM tariffs Fund asked 500 On same rate right ministers If 64 nil Total management restrictions crop 83 effective Community producers available Securities palm output mark stimulate *UNK* action prices situation 8 generally traders ended own reaffirmed making much NEW a better with do SEES being reduce into Stoltenberg shareholders Dealers 2ND Chrysler 7 maize he may political market noted Plc businesses so particularly 44 members continuing payments told which Group tender make *UNK* currently drop asked little paper far key income prevent AS 61 shrs put Sales session confirmed are field include include expenses . firm 76 pretax Italy 87 outlook compar

## Spelling Corection using the Bigram Model with Beam Search

In [18]:
#query (iv) Spelling correction using the bigram model with Beam Search Decoder
#------------------

# Define a function to generate candidate words given the last word and vocabulary
def generate_candidates(last_word, vocabulary):
    # Given the last word, generate possible next words
    # Calculate edit distances and select candidates with edit distances > 0
    edit_distances = [(w, nltk.edit_distance(last_word, w)) for w in vocabulary if nltk.edit_distance(last_word, w) > 0]
    # Sort candidates by edit distance
    cand_words = sorted(edit_distances, key=lambda x: x[1])[:5]
    # If the last word is already in the vocabulary, add it as a candidate with an edit distance of 0
    if last_word in vocabulary:
        cand_words = [(last_word, 0)] + cand_words
    return cand_words

# Define a function to score a word sequence using the bigram model
def score(prev_word, last_word, bigram_model, vocabulary):
    # Calculate the probability of the word sequence using the bigram model
    prev_word_index = vocabulary.index(prev_word)
    last_word_index = vocabulary.index(last_word[0])
    words_prob = bigram_model[prev_word_index][last_word_index]
    if words_prob < 1e-40:
        words_prob = 1e-40
    # Calculate the score as the log probability of the word sequence
    return log(words_prob * 10**41) + log(1.0 * 10**41 / (last_word[1] + 1.0))

# Define a beam search decoder function to correct the sentence
def beam_search_decode(initial_state, beam_width, vocabulary, bigram_model, l=0.5):
    candidates = [(initial_state, 1.0)]
    temp_vocabulary = vocabulary.copy()
    temp_vocabulary.append('*start*')  # Add the "*start*" token to the vocabulary

    for w in range(1, len(initial_state)):  # Select each word
        temp_candidates = []
        for candidate in candidates:
            new_candidates_words = []
            for c in generate_candidates(candidate[0][w], temp_vocabulary):  # Generate candidate words
                new_prob = log(candidate[1] * 10**41) + score(candidate[0][w - 1], c, bigram_model, temp_vocabulary)  # Calculate score
                temp_candidate = candidate[0].copy()
                temp_candidate[w] = c[0]
                new_candidates_words.append((temp_candidate, new_prob))  # Save new sentence with probability
            new_candidates_words = sorted(new_candidates_words, key=lambda x: x[1], reverse=True)
            temp_candidates += new_candidates_words  # Save each sentence

        candidates = sorted(temp_candidates, key=lambda x: x[1], reverse=True)[:beam_width]  # Keep the best sentences

    best_sequence, best_prob = max(candidates, key=lambda x: x[1])
    return best_sequence, best_prob


beam_width = 2

# Example usage:
# Initial sentence with a spelling error
initial_state = ['*start*', 'Thizz', 'yar', 'is', 'expeced', 'to', 'icrese', '*end*']

# Apply beam search decoding to correct the sentence
best_sequence, best_prob = beam_search_decode(initial_state, beam_width, vocabulary, bigram_model)

# Print the corrected sentence and its probability
print("Best sequence:", best_sequence[1:], best_prob)  # Excluding the "*start*" token

initial_state2 = ['*start*', 'Tho', 'fil', 'wa', 'vry', 'bring', '*end*']

# Apply beam search decoding to correct the sentence
best_sequence, best_prob = beam_search_decode(initial_state2, beam_width, vocabulary, bigram_model)

# Print the corrected sentence and its probability
print("Best sequence:", best_sequence[1:], best_prob)  # Excluding the "*start*" token

Best sequence: ['The', 'year', 'is', 'expected', 'to', 'increase', '*end*'] 277.4289810926972
Best sequence: ['The', 'oil', 'a', 'very', 'being', '*end*'] 277.07392493104504


## Artificial Test Corpus with errors
Testing & Evaluating the previous spelling correction method using the metrics:
- Word Error Rate
- Character Error Rate 

In [40]:
#query (v)
#------------------
import random
random.seed(10)

# Define a function to randomly shuffle characters in a word
def random_suffler(word):
    list_word = list(word)
    rc_word = (random.randint(0, len(word) - 1))
    rc1 = random.randint(65, 90)  # ASCII values for uppercase letters
    rc2 = random.randint(97, 122)  # ASCII values for lowercase letters
    list_word[rc_word] = chr(random.choice([rc1, rc2]))
    return "".join(list_word)

# Create an artificial test corpus by introducing spelling errors
artificial_test_corpus = [list(sentence) for sentence in test_corpus]
for sentence in artificial_test_corpus:
    for word in range(len(sentence)):
        sentence[word] = random_suffler(sentence[word]) if random.random() >= 0.5 else sentence[word]


#query (vi)
#------------------

# Apply beam search decoding to correct the artificial test corpus
for sentence in tqdm(range(len(artificial_test_corpus)), desc="Spelling correction with beam search"):
    artificial_test_corpus[sentence] = beam_search_decode(['*start*'] + artificial_test_corpus[sentence] + ['*end*'], beam_width, vocabulary, bigram_model)[0][1:-1]

#%pip install evaluate
from evaluate import load

# Compute Word Error Rate (WER) and Character Error Rate (CER)
wer = load("wer")
cer = load("cer")

total_wer = 0.
total_cer = 0.

for sent in tqdm(range(len(artificial_test_corpus)), desc="Calculate WER and CER for each sentence"):
    pred = [" ".join(artificial_test_corpus[sent])]
    refr = [" ".join(test_corpus[sent])]
    total_wer += wer.compute(predictions=pred, references=refr)
    total_cer += cer.compute(predictions=pred, references=refr)

print("WER: ", total_wer / len(artificial_test_corpus))
print("CER: ", total_cer / len(artificial_test_corpus))

Calculate WER and CER for each sentence: 100%|██████████████████████████████████████| 600/600 [00:03<00:00, 161.30it/s]

WER:  0.22509975700557588
CER:  0.0731300603259554


In [62]:
print("Reference sentence: ", " ".join(test_corpus[0]))
print("Corrected sentence: ", " ".join(artificial_test_corpus[0]))

print("\nReference sentence: ", " ".join(test_corpus[65]))
print("Corrected sentence: ", " ".join(artificial_test_corpus[65]))

Reference sentence:  Some analysts said American *UNK* could use capital since it plans to expand *UNK* .
Corrected sentence:  Some analysts said chemical *UNK* would use capital since it plans to expand *UNK* :

Reference sentence:  *UNK* *UNK* , he said , *UNK* a *UNK* *UNK* for *UNK* crude oil while *UNK* stability of supply to *UNK* .
Corrected sentence:  *UNK* *UNK* , he said , *UNK* : *UNK* *UNK* for *UNK* crude oil while *UNK* stability of supply to *UNK* .


In [67]:
print("Reference sentence: ", " ".join(test_corpus[30]))
print("Corrected sentence: ", " ".join(artificial_test_corpus[30]))

print("\nReference sentence: ", " ".join(test_corpus[75]))
print("Corrected sentence: ", " ".join(artificial_test_corpus[75]))

Reference sentence:  *UNK* GROUP SAYS IT *UNK* *UNK* TALKS ON *UNK* *UNK* OF *UNK* *UNK*
Corrected sentence:  *UNK* CORP SAYS IT *UNK* *UNK* SAYS OF *UNK* *UNK* OF *UNK* *UNK*

Reference sentence:  *UNK* the *UNK* group , another major *UNK* *UNK* , *UNK* , *UNK* , *UNK* *UNK* *UNK* , has also said he has had talks about increasing his stake in the company , taking part in a takeover *UNK* , or *UNK* one *UNK* .
Corrected sentence:  *UNK* the *UNK* group B another major *UNK* *UNK* : *UNK* , *UNK* : *UNK* *UNK* *UNK* : Pay also said the has had sales about increasing his stake in the company , saying part in a takeover *UNK* : of *UNK* on *UNK* :
